Tugas 1: Buat DataFrame sederhana di Spark dan eksplorasi beberapa fungsi dasar yang tersedia.


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [
    ('James',   'Sales',     3000, 30, 5, 'M'),
    ('Michael', 'Sales',     4600, 40, 12, 'M'),
    ('Robert',  'Sales',     4100, 36, 10, 'M'),
    ('Maria',   'Finance',   3000, 29, 3, 'F'),
    ('Jen',     'Finance',   3900, 32, 6, 'F'),
    ('Jeff',    'Marketing', 3000, 28, 2, 'M'),
    ('Kumar',   'Marketing', 2000, 25, 1, 'M'),
    ('Saif',    'IT',        3300, 35, 8, 'M'),
    ('Rina',    'IT',        4000, 31, 7, 'F'),
    ('Adi',     'HR',        2500, 26, 2, 'M')
]

columns = ['EmployeeName', 'Department', 'Salary', 'Age', 'Experience', 'Gender']
df = spark.createDataFrame(data, schema=columns)
# Fungsi dasar
df.show()
df.printSchema()
print("Jumlah baris:", df.count())
print("Jumlah kolom:", len(df.columns))
df.describe().show()

+------------+----------+------+---+----------+------+
|EmployeeName|Department|Salary|Age|Experience|Gender|
+------------+----------+------+---+----------+------+
|       James|     Sales|  3000| 30|         5|     M|
|     Michael|     Sales|  4600| 40|        12|     M|
|      Robert|     Sales|  4100| 36|        10|     M|
|       Maria|   Finance|  3000| 29|         3|     F|
|         Jen|   Finance|  3900| 32|         6|     F|
|        Jeff| Marketing|  3000| 28|         2|     M|
|       Kumar| Marketing|  2000| 25|         1|     M|
|        Saif|        IT|  3300| 35|         8|     M|
|        Rina|        IT|  4000| 31|         7|     F|
|         Adi|        HR|  2500| 26|         2|     M|
+------------+----------+------+---+----------+------+

root
 |-- EmployeeName: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- Experience: long (nullable = true)
 |-- Gender: string (nullable

Tugas 2: Gunakan operasi filter, select, groupBy untuk mengekstrak informasi dari data, serta lakukan agregasi data untuk mendapatkan insight tentang dataset menggunakan perintah seperti mean, max, sum.

In [ ]:
from pyspark.sql import functions as F

# Pilih kolom tertentu
df.select('EmployeeName', 'Salary').show()

# Filter gaji > 3000
df.filter(df['Salary'] > 3500).show()

# Agregasi
df.groupBy('Department').agg(
    F.avg('Salary').alias('Avg_Salary'),
    F.sum('Salary').alias('Total_Salary'),
    F.max('Salary').alias('Max_Salary'),
    F.count('EmployeeName').alias('Jumlah_Pegawai')
).show()


+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
|         Jen|  3900|
|        Jeff|  3000|
|       Kumar|  2000|
|        Saif|  3300|
|        Rina|  4000|
|         Adi|  2500|
+------------+------+

+------------+----------+------+---+----------+------+
|EmployeeName|Department|Salary|Age|Experience|Gender|
+------------+----------+------+---+----------+------+
|     Michael|     Sales|  4600| 40|        12|     M|
|      Robert|     Sales|  4100| 36|        10|     M|
|         Jen|   Finance|  3900| 32|         6|     F|
|        Rina|        IT|  4000| 31|         7|     F|
+------------+----------+------+---+----------+------+

+----------+----------+------------+----------+--------------+
|Department|Avg_Salary|Total_Salary|Max_Salary|Jumlah_Pegawai|
+----------+----------+------------+----------+--------------+
|     Sales|    3900.0|       11700|      4600|             3

Tugas 3: Eksplorasi bagaimana mengolah tipe data kompleks dalam Spark DataFrames.

In [ ]:
from pyspark.sql.functions import array
from pyspark.sql.functions import create_map, lit
from pyspark.sql.functions import struct

#ArrayType
df_array = df.withColumn("Skills", array("Department" , "Experience"))
df_array.show(truncate=False)

#MapType
df_map = df.withColumn("Info", create_map(lit("Salary"), df["Salary"], lit("Age"), df["Age"]))
df_map.show(truncate=False)

#StructType
df_struct = df.withColumn("Profile", struct("Age", "Experience", "Gender"))
df_struct.show(truncate=False)

+------------+----------+------+---+----------+------+--------------+
|EmployeeName|Department|Salary|Age|Experience|Gender|Skills        |
+------------+----------+------+---+----------+------+--------------+
|James       |Sales     |3000  |30 |5         |M     |[Sales, 5]    |
|Michael     |Sales     |4600  |40 |12        |M     |[Sales, 12]   |
|Robert      |Sales     |4100  |36 |10        |M     |[Sales, 10]   |
|Maria       |Finance   |3000  |29 |3         |F     |[Finance, 3]  |
|Jen         |Finance   |3900  |32 |6         |F     |[Finance, 6]  |
|Jeff        |Marketing |3000  |28 |2         |M     |[Marketing, 2]|
|Kumar       |Marketing |2000  |25 |1         |M     |[Marketing, 1]|
|Saif        |IT        |3300  |35 |8         |M     |[IT, 8]       |
|Rina        |IT        |4000  |31 |7         |F     |[IT, 7]       |
|Adi         |HR        |2500  |26 |2         |M     |[HR, 2]       |
+------------+----------+------+---+----------+------+--------------+

+------------+-----

Tugas 4: Implementasikan window function untuk menghitung running totals atau rangkings.

In [ ]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

# Buat window per department, diurutkan berdasarkan Salary
windowSpec = Window.partitionBy("Department").orderBy(df["Salary"].desc())

# Ranking gaji per department
df.withColumn("RankInDept", F.rank().over(windowSpec)).show()

# Running total gaji per department
df.withColumn("RunningTotal", F.sum("Salary").over(windowSpec)).show()


def categorize_salary(salary):
    if salary > 4000:
        return "High"
    elif salary > 3000:
        return "Medium"
    else:
        return "Low"

# Daftarkan sebagai UDF
categorize_salary_udf = udf(categorize_salary, StringType())

# Gunakan di DataFrame
df.withColumn("SalaryCategory", categorize_salary_udf("Salary")).show()


+------------+----------+------+---+----------+------+----------+
|EmployeeName|Department|Salary|Age|Experience|Gender|RankInDept|
+------------+----------+------+---+----------+------+----------+
|         Jen|   Finance|  3900| 32|         6|     F|         1|
|       Maria|   Finance|  3000| 29|         3|     F|         2|
|         Adi|        HR|  2500| 26|         2|     M|         1|
|        Rina|        IT|  4000| 31|         7|     F|         1|
|        Saif|        IT|  3300| 35|         8|     M|         2|
|        Jeff| Marketing|  3000| 28|         2|     M|         1|
|       Kumar| Marketing|  2000| 25|         1|     M|         2|
|     Michael|     Sales|  4600| 40|        12|     M|         1|
|      Robert|     Sales|  4100| 36|        10|     M|         2|
|       James|     Sales|  3000| 30|         5|     M|         3|
+------------+----------+------+---+----------+------+----------+

+------------+----------+------+---+----------+------+------------+
|Employ

Tugas 5:

1.   Unduh dataset besar dari Kaggle atau sumber lainnya.
2. Input data csv yang telah di download, kemudian load dan simpan data ke dalam pyspark.
1.   Setelah data berhasil di load menggunakan pyspark, lakukan manipulasi data untuk memperoleh informasi yang dibutuhkan





In [31]:
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, year, stddev, lag, col, round, first, last
from pyspark.sql.window import Window

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Teknologi-Informasi/daily_forex_rates.csv'

spark = SparkSession.builder.appName('Data-Forex').getOrCreate()
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()

#Filter data
currencies = ["IDR", "JPY", "GBP", "USD", "XAU"]
df_filtered = df.filter(df.currency.isin(currencies))
df_from2022 = df_filtered.filter(year("date") >= 2022) \
                         .withColumn("year", year("date"))

#Rata-rata kurs per tahun
df_from2022.groupBy("currency", "year")\
          .agg(avg("exchange_rate").alias("avg_exchange_rate_EUR"))\
          .orderBy("year").show()

#Mencari mata uang paling fluktuatif
df_from2022.groupBy("currency")\
          .agg(stddev("exchange_rate")\
          .alias("stddev_exchange_rate_EUR"))\
          .orderBy("stddev_exchange_rate_EUR", ascending=False)\
          .show()

#Korelasi antara USD dan XAU
df_usd_xau = df_from2022.filter(df.currency.isin(["USD", "XAU"]))

window_spec = Window.partitionBy("currency").orderBy("date")

df_usd_xau = df_usd_xau.withColumn("prev_rate", lag("exchange_rate")\
            .over(window_spec)) \
            .withColumn("daily_return",
                        (col("exchange_rate") - col("prev_rate")) / col("prev_rate"))

df_pivot = df_usd_xau.groupBy("date").pivot("currency")\
            .agg({"daily_return":"first"}) \
            .withColumnRenamed("USD", "usd_return") \
            .withColumnRenamed("XAU", "xau_return")

corr = df_pivot.stat.corr("usd_return", "xau_return")
print(f"Korelasi USD vs XAU: {corr:.3f} \n")

#Hitung rata-rata daily return
df_yearly_return = df_from2022.groupBy("currency", "year") \
    .agg(
        first("exchange_rate").alias("rate_start"),
        last("exchange_rate").alias("rate_end")
    )

df_yearly_return = df_yearly_return.withColumn(
    "yearly_return_percent",
    round((col("rate_end") - col("rate_start")) / col("rate_start") * 100, 3)
)

df_yearly_return.select("currency", "year", "yearly_return_percent") \
                .orderBy("year", "currency") \
                .show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
root
 |-- currency: string (nullable = true)
 |-- base_currency: string (nullable = true)
 |-- currency_name: string (nullable = true)
 |-- exchange_rate: double (nullable = true)
 |-- date: date (nullable = true)

+--------+----+---------------------+
|currency|year|avg_exchange_rate_EUR|
+--------+----+---------------------+
|     IDR|2022|   15621.496153846154|
|     GBP|2022|    0.852446692307692|
|     JPY|2022|            138.06405|
|     USD|2022|    1.053234115384615|
|     XAU|2023|  5.40111111111111E-4|
|     JPY|2023|   152.43882649280584|
|     IDR|2023|   16488.242575284174|
|     GBP|2023|   0.8693861546762592|
|     USD|2023|   1.0819061690647485|
|     IDR|2024|   17157.253934377044|
|     XAU|2024| 4.571671232876722E-4|
|     JPY|2024|    163.8818509153005|
|     GBP|2024|   0.8469286393442617|
|     USD|2024|    1.082318587431693|
|     GBP|